ReActChatFormatter.from_context is deprecated, please use `from_defaults` instead.


Enter a prompt (q to quit):  tell me about 2 common nutritional deficiencies


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: defic_data
Action Input: {'input': 'common nutritional deficiencies'}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Observation: The common nutritional deficiencies identified in the context are for vitamin B6, iron, and vitamin D. These deficiencies varied by age, gender, or race/ethnicity in the U.S. population, with certain groups like non-Hispanic black and Mexican-American individuals more likely to be deficient in these

Enter a prompt (q to quit):  why is vitamin D important?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Thought: The user is asking about the importance of vitamin D.
Action: defic_data
Action Input: {'input': 'vitamin D'}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Observation: Vitamin D is essential for good bone health, muscle strength, and may help protect against cancer and type 2 diabetes. It is naturally found in only a few foods and is commonly added to milk and other foods in the United States. The body forms vitamin D in the skin with the help of ultraviolet light from the sun. Doctors use the form of vitamin D called 25-hydroxyv

KeyboardInterrupt: Interrupted by user

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
import os
import openai

os.environ["OPENAI_API_KEY"] = ""


In [16]:
from dotenv import load_dotenv
load_dotenv()

import os
import pandas as pd
from llama_index.core.query_engine import PandasQueryEngine
#from note_engine import note_engine
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core import PromptTemplate

# Detailed instructions for the usage of the generated Python code with pandas
instruction_str = """
1. Convert the query to executable Python code using Pandas.
2. The final line of code should be a Python expression that can be called with the `eval()` function.
3. Ensure the code represents a solution to the query regarding nutrition data analysis.
4. PRINT ONLY THE EXPRESSION, ensuring it is syntactically correct and does not include extraneous quotes.
5. Focus on accuracy and relevance to the query, pulling directly from nutritional data sets.
"""

# Template for prompting which includes dynamic data display and tailored instructions
new_prompt = PromptTemplate(
    """\
    You are working with a pandas dataframe named `df` that includes nutritional data similar to the CDC's Second Nutrition Report.
    Here is a preview of the dataframe (`df.head()`):
    {df_str}

    Please follow these detailed instructions:
    {instruction_str}
    Query: {query_str}

    Write the Python expression that answers the query:
    """
)

# Context description to inform the agent about its operational purpose
context = """Purpose: Assist users by providing accurate information and analysis based on nutrition data.
            This agent aids in interpreting population statistics and specifics about nutrient deficiencies
            and health outcomes from structured data sources like the CDC's NHANES nutrition reports."""



# json_file_paths = [
#     os.path.join("data", "foundation.json"),
#     os.path.join("data", "legacy.json"),
#     os.path.join("data", "survey.json")
# ]

# dataframes = [pd.read_json(path) for path in json_file_paths]
# combined_df = pd.concat(dataframes, ignore_index=True)

# population_query_engine = PandasQueryEngine(
#     df=combined_df, verbose=True, instruction_str=instruction_str
# )
#population_query_engine.update_prompts({"pandas_prompt": new_prompt})

tools = [
    QueryEngineTool(
        query_engine=defic_engine,
        metadata=ToolMetadata(
            name="defic_data",
            description="this gives detailed information about common nutritional deficiencies",
        ),
    ),
]

llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = ReActAgent.from_tools(tools, llm=llm, verbose=True, context=context)

while (prompt := input("Enter a prompt (q to quit): ")) != "q":
    result = agent.query(prompt)
    print(result)

In [7]:
!pip install genson

import genson
import json

with open('legacy.json', 'r') as file:
    json_data = json.load(file)

  Created wheel for genson: filename=genson-1.2.2-py2.py3-none-any.whl size=21262 sha256=4cfb1445051dbd0cd554a39815349bd8a4ae5edeb5dc36d3a7a1a77fbe621f95
  Stored in directory: /Users/Varun/Library/Caches/pip/wheels/3c/d7/98/a29f29e3861092e784c4c4994b0b30304c79da1bcbc8284af1
Successfully built genson
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.


In [36]:
builder = genson.SchemaBuilder()
builder.add_object(json_data)
#json_schema = builder.to_schema()
json_schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "description": "Schema for the nutritional information of various food items as per the SR Legacy data format",
    "type": "object",
    "properties": {
        "SRLegacyFoods": {
            "description": "An array of food items containing detailed nutritional information and other relevant data.",
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "foodClass": {
                        "description": "The classification of the food item (e.g., FinalFood, Raw, etc.).",
                        "type": "string"
                    },
                    "description": {
                        "description": "A detailed description of the food item, including brand and physical state.",
                        "type": "string"
                    },
                    "foodNutrients": {
                        "description": "Array of nutrients found in the food item.",
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "type": {
                                    "description": "Type of nutrient record.",
                                    "type": "string"
                                },
                                "id": {
                                    "description": "Unique identifier for the nutrient record.",
                                    "type": "integer"
                                },
                                "nutrient": {
                                    "description": "Details about the nutrient type.",
                                    "type": "object",
                                    "properties": {
                                        "id": {
                                            "description": "Identifier for the type of nutrient.",
                                            "type": "integer"
                                        },
                                        "number": {
                                            "description": "Code number for the nutrient.",
                                            "type": "string"
                                        },
                                        "name": {
                                            "description": "Name of the nutrient.",
                                            "type": "string"
                                        },
                                        "rank": {
                                            "description": "Ranking used to order nutrients in displays.",
                                            "type": "integer"
                                        },
                                        "unitName": {
                                            "description": "Unit of measure for the nutrient amount.",
                                            "type": "string"
                                        }
                                    },
                                    "required": ["id", "name", "number", "rank", "unitName"]
                                },
                                "dataPoints": {
                                    "description": "The number of data points used to derive the nutrient value.",
                                    "type": "integer"
                                },
                                "foodNutrientDerivation": {
                                    "description": "How the nutrient value was derived.",
                                    "type": "object",
                                    "properties": {
                                        "code": {
                                            "description": "Code indicating the derivation method.",
                                            "type": "string"
                                        },
                                        "description": {
                                            "description": "Description of the derivation method.",
                                            "type": "string"
                                        },
                                        "foodNutrientSource": {
                                            "description": "Source of the nutrient data.",
                                            "type": "object",
                                            "properties": {
                                                "id": {
                                                    "description": "Identifier for the source of the data.",
                                                    "type": "integer"
                                                },
                                                "code": {
                                                    "description": "Code representing the source.",
                                                    "type": "string"
                                                },
                                                "description": {
                                                    "description": "Description of the source.",
                                                    "type": "string"
                                                }
                                            },
                                            "required": ["id", "code", "description"]
                                        }
                                    },
                                    "required": ["code", "description", "foodNutrientSource"]
                                },
                                "amount": {
                                    "description": "Amount of the nutrient in the food item.",
                                    "type": "number"
                                }
                            },
                            "required": ["type", "id", "nutrient", "amount", "dataPoints", "foodNutrientDerivation"]
                        }
                    },
                    "foodCategory": {
                        "description": "Category of the food item.",
                        "type": "object",
                        "properties": {
                            "description": {
                                "description": "Description of the food category.",
                                "type": "string"
                            }
                        },
                        "required": ["description"]
                    },
                    "fdcId": {
                        "description": "FoodData Central identifier for the food item.",
                        "type": "integer"
                    },
                    "dataType": {
                        "description": "Type of data entry (e.g., SR Legacy).",
                        "type": "string"
                    },
                    "publicationDate": {
                        "description": "Date when the food item was published in the database.",
                        "type": "string"
                    },
                    "isHistoricalReference": {
                        "description": "Indicates if the item is used as a historical reference.",
                        "type": "boolean"
                    }
                },
                "required": ["foodClass", "description", "foodNutrients", "foodCategory", "fdcId", "dataType", "publicationDate", "isHistoricalReference"]
            }
        }
    },
    "required": ["SRLegacyFoods"]
}


print(json_schema)

{'$schema': 'http://json-schema.org/draft-07/schema#', 'description': 'Schema for the nutritional information of various food items as per the SR Legacy data format', 'type': 'object', 'properties': {'SRLegacyFoods': {'description': 'An array of food items containing detailed nutritional information and other relevant data.', 'type': 'array', 'items': {'type': 'object', 'properties': {'foodClass': {'description': 'The classification of the food item (e.g., FinalFood, Raw, etc.).', 'type': 'string'}, 'description': {'description': 'A detailed description of the food item, including brand and physical state.', 'type': 'string'}, 'foodNutrients': {'description': 'Array of nutrients found in the food item.', 'type': 'array', 'items': {'type': 'object', 'properties': {'type': {'description': 'Type of nutrient record.', 'type': 'string'}, 'id': {'description': 'Unique identifier for the nutrient record.', 'type': 'integer'}, 'nutrient': {'description': 'Details about the nutrient type.', 'ty

In [70]:
from llama_index.core import StorageContext, VectorStoreIndex, load_index_from_storage
from llama_index.readers.file import PDFReader


def get_index(data, index_name):
    index = None
    if not os.path.exists(index_name):
        print("building index", index_name)
        index = VectorStoreIndex.from_documents(data, show_progress=True)
        index.storage_context.persist(persist_dir=index_name)
    else:
        index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=index_name)
        )

    return index


pdf_path = "ting.pdf"
defic_pdf = PDFReader().load_data(file=pdf_path)
defic_index = get_index(defic_pdf, "deficiency")
defic_engine = defic_index.as_query_engine()

building index deficiency


Parsing nodes:   0%|          | 0/4 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [ ]:
tools = [
    QueryEngineTool(
        query_engine=defic_engine,
        metadata=ToolMetadata(
            name="defic_data",
            description="this gives detailed information about common nutritional deficiencies",
        ),
    ),
]

In [64]:
!pip install llama-index-readers-file


You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.


In [48]:
from llama_index.llms.openai import OpenAI
from llama_index.core.indices.struct_store import JSONQueryEngine

llm = OpenAI(model="gpt-3.5-turbo-0125")

nl_query_engine = JSONQueryEngine(
    json_value=json_data,
    json_schema=json_schema,
    llm=llm,
)

In [12]:
from IPython.display import Markdown, display

In [47]:
nl_response = nl_query_engine.query(
    "Give description for 3 foods and their nutrients",
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.752180 seconds
Retrying request to /chat/completions in 0.752180 seconds


KeyboardInterrupt: 

In [43]:
display(
    Markdown(f"<h1>Natural language Response</h1><br><b>{nl_response}</b>")
)

<h1>Natural language Response</h1><br><b>Pillsbury Golden Layer Buttermilk Biscuits, Artificial Flavor, refrigerated dough, Pillsbury, Cinnamon Rolls with Icing, refrigerated dough, Kraft Foods, Shake N Bake Original Recipe, Coating for Pork, dry, George Weston Bakeries, Thomas English Muffins, Waffles, buttermilk, frozen, ready-to-heat, Waffle, buttermilk, frozen, ready-to-heat, toasted, Waffle, buttermilk, frozen, ready-to-heat, microwaved, Waffle, plain, frozen, ready-to-heat, microwave, Pie Crust, Cookie-type, Graham Cracker, Ready Crust, Pie Crust, Cookie-type, Chocolate, Ready Crust</b>

In [22]:
!pip install jsonpath-ng

     |████████████████████████████████| 49 kB 5.7 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.
